# Classification ML models
### Two models were trained to classify between class I and class II:
### **Support Vector Machine (SVM):** supervised learning
### **Gaussian Mixture:** unsupervised learning

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
####################################
from pymatgen.core import Composition
####################################
from matplotlib import pyplot as plt
from matplotlib import patches, cm, colors
from matplotlib import patheffects as pe
import matplotlib.ticker as ticker
from matplotlib.transforms import ScaledTranslation
from matplotlib.markers import MarkerStyle
####################################
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.svm import SVC, LinearSVC
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture

In [2]:
plt.rcParams.update({"figure.facecolor":(1.0, 1.0, 1.0, 0.0),
                     "axes.facecolor":(1.0, 1.0, 1.0, 0.0)})
plt.rcParams["font.family"] = 'Helvetica'
plt.rcParams["font.size"] = 30
plt.rcParams['mathtext.fontset'] = 'cm'
plt.rcParams['mathtext.it'] = 'Helvetica'
plt.rcParams['mathtext.rm'] = 'Helvetica'
plt.rcParams['figure.figsize'] = (10,8)

In [3]:
adf = pd.read_excel('C:/Users/mualemy5/Desktop/Self-healing/Supporting_Tables.xlsx', sheet_name='S1 - Full Dataset')

In [4]:
adf = adf.loc[[i for i, r in adf.iterrows() if r.ionicity!=r.ionicity or (r.ionicity>=0) or
               any(a in r.chemical_formula for a in ['YSZ', 'La1.'])]]#no negative ionicity

In [5]:
df = adf.groupby(['chemical_formula', 'space_group', 'cation_coordination']).mean(numeric_only=True).reset_index()

In [6]:
evac = pd.read_csv('C:/Users/mualemy5/SH_bond_properties/oxygen_vacancies_enthalpies.csv')

In [7]:
haps = [] #Halide Perovskites (green)
alkali_halides = [] #Alkali Halides (yellow)
coinage_halides = [] #Coinage Halides (orange)
chalcohalides = [] #Chalcohalides (purple)
chalcopyrites = [] #Chalcopyrites (red)
CSOFCs = [] #Ceramics for solid oxide fuel cells (blue)

for index, row in df.iterrows():
    if 'YSZ' in row.chemical_formula or 'La1.' in row.chemical_formula:
        CSOFCs.append(index)
        continue
        
    comp = Composition(row.chemical_formula)
    elements = list(comp.as_dict().keys())
    raw_cations = [a.split("', '")[0].replace("'", '').replace('(', '').replace('+', '')
                   for a in row.cation_coordination.split("), (")]
    cations = [''.join(c for c in cat if not c.isdigit()) for cat in raw_cations] #species
    cations = list(set(cations))
    ox_cations = []#species+state
    for cat in raw_cations: 
        if cat[-1].isdigit():
            ox_cations.append(cat)
        else:
            ox_cations.append(cat+'1')
    ox_cations = list(set(ox_cations))
    anions = [an for an in elements if an not in cations] #species

    #Halide Perovskites#
    #Definition is based on composition - ABX3 (including A being MA).
    if (any([x in anions for x in ['Cl', 'Br', 'I']]) and len(comp)>=3 and 'Octahedron' in str(row.cation_coordination) and
        all([comp[x]==3 for x in [x for x in ['Cl', 'Br', 'I'] if x in comp]]) and
        all([comp[x]==1 or comp[x]==6 for x in [x for x in comp.as_dict().keys() if x not in ['Cl', 'Br', 'I']]])):
        if row.chemical_formula not in ['CaInBr3', 'CaTlCl3', 'CaTlBr3']: #proved to not be a perovskite
            haps.append(index)

    #Alkali Halides#
    #Definition is based on composition - alkali metal and an halide 
    if (any([x in anions for x in ['Cl', 'Br', 'I']]) and any([x in comp for x in ['Li', 'Na', 'K', 'Rb', 'Cs']])
        and len(comp)==2):
        alkali_halides.append(index)

    #Coinage Halides#
    #Definition is based on composition - coinage metal and an halide 
    if (any([x in anions for x in ['Cl', 'Br', 'I']]) and any([x in comp for x in ['Cu', 'Ag', 'Au']])
        and len(comp)==2):
        coinage_halides.append(index)

    #Chalcohalides#
    #Definition is based on composition - halide and chalcogenide (anions!)
    if any([x in anions for x in ['Cl', 'Br', 'I']]) and any([x in comp for x in ['S', 'Se', 'Te']]) and len(comp)>2:
        if row.cation_coordination==row.cation_coordination:
            if all([all([x!=''.join(aa for aa in a.split("'")[1] if not aa.isdigit())[:-1]
                         for a in row.cation_coordination.split(', ')[::3]])
                   for x in ['S', 'Se', 'Te']]): #those are not cations
                chalcohalides.append(index)

    #Possible CSOFCs#
    #Definition is based on estimated energy of vacancy formation
    if anions==['O']:
        Evacs = []
        weights = []
        if all([cat in evac.atom.tolist() for cat in cations]): #First, we can say something about the atom
            for cat in cations:
                if len(cat)==2: #2-letter element
                    ox = [a[-1] for a in ox_cations if cat==a[:2]]
                else: #1-letter element
                    ox = [a[-1] for a in ox_cations if len(a)==2 and cat==a[0]]
                ox = list(set(ox)) #all ox states of atom
                if len(ox)>=2: #more than two- we add automatically to not lose a point
                    CSOFCs.append(index)
                elif len(ox)>0: #there is any
                    ox = int(ox[0])
                    Evac_val = evac.loc[evac.atom==cat].loc[evac.oxidation_state==ox].Evac.tolist()
                    if len(Evac_val)>0:
                        Evacs.append(comp[cat]*float(Evac_val[0])) #Evac of atom
                        weights.append(comp[cat])
            if len(Evacs)>0:
                if np.sum(Evacs)>2*sum(weights) and np.sum(Evacs)<5*sum(weights): #weighted average
                    CSOFCs.append(index)

In [ ]:
plt.figure(figsize=(10,6))
fi_df = df.loc[[i for i, r in df.iterrows() if r.ionicity<=1 and r.hardness>1e-2 or any(a in r.chemical_formula for a in ['YSZ', 'La1.'])]]

norm = colors.Normalize(vmin=fi_df.ionicity.min(), vmax=fi_df.ionicity.max())
cmap = cm.get_cmap('gist_rainbow_r')

YBCO_entry = fi_df.loc[fi_df.chemical_formula=='Ba2YCu3O7']

plt.scatter(YBCO_entry.hardness, YBCO_entry.e_ionic/YBCO_entry.e_electronic, s=110, color='none', edgecolor='blue')
plt.scatter(fi_df.hardness, fi_df.e_ionic/fi_df.e_electronic, c=fi_df.ionicity, s=50, cmap=cmap, alpha=0.8, norm=norm)
for val in [(5.4+5+5.1+4.7)/4, 8.5, 0.53, 7.81, 8.7, 3.01, 10.28, 14]:
    plt.scatter(val, YBCO_entry.e_ionic/YBCO_entry.e_electronic, s=110, color='none', edgecolor='blue')
    plt.scatter(val, YBCO_entry.e_ionic/YBCO_entry.e_electronic, c=YBCO_entry.ionicity, s=50, alpha=0.8, cmap=cmap, norm=norm)

plt.colorbar(label='Ionicity')
ysz = fi_df.loc[fi_df.chemical_formula=='8YSZ']
plt.scatter(ysz.hardness[0], list(ysz.e_ionic/ysz.e_electronic)[0], s=50, color='black')

for index, row in df.iterrows():
    if row.ionicity == row.ionicity and row.hardness==row.hardness:
        #if row.chemical_formula.endswith('S'):
        #plt.annotate(row.chemical_formula.replace('CH3NH3', 'MA'), (row.hardness, row.e_ionic/row.e_electronic), ha='center', va='center', fontsize=7)
        continue

cis = fi_df.loc[fi_df.chemical_formula=='InCuSe2']
cgs = fi_df.loc[fi_df.chemical_formula=='GaCuSe2']

plt.annotate('', xy=(cis.hardness, cis.e_ionic/cis.e_electronic*0.95), xytext=(2, 10),
                arrowprops=dict(color='red', arrowstyle='->', linewidth=2, alpha=.75))
plt.annotate('', xy=(cgs.hardness, cgs.e_ionic/cgs.e_electronic*0.95), xytext=(2, 10),
                arrowprops=dict(color='red', arrowstyle='->', linewidth=2, alpha=.75))
plt.annotate('CIGS', (2.2,9), ha='center', va='bottom', fontsize=18, c='red')


plt.annotate('YSZ', xy=(ysz.hardness[0], list(ysz.e_ionic/ysz.e_electronic)[0]*0.95), xytext=(38, 4),
                arrowprops=dict(color='blue', arrowstyle='->', linewidth=2, alpha=.75), ha='center', va='bottom', fontsize=16, c='blue')

plt.annotate('YBCO', xy=(YBCO_entry.hardness*0.96, YBCO_entry.e_ionic/YBCO_entry.e_electronic*0.89), xytext=(1e-1, 4e-2),
                arrowprops=dict(color='blue', arrowstyle='->', linewidth=2, alpha=.75), ha='center', va='bottom', fontsize=16, c='blue')

plt.annotate('Pnictides', (2e1, 2.5e-2), fontsize=16, color='fuchsia', ha='center', va='center')
plt.annotate('(III-V Semiconductors)', (2e1, 1.5e-2), fontsize=12, color='fuchsia', ha='center', va='center')
ell_tform = ScaledTranslation(4.5e0, 8.5e-2, plt.gca().transScale) + plt.gca().transLimits + plt.gca().transAxes
el1 = patches.Ellipse(xy=(0, 0), width=1.2, height=3e-1, angle=60, facecolor='fuchsia', alpha=.25, edgecolor='none', transform=ell_tform, zorder=-100)

plt.annotate('Halide\nPerovskites', (1.5e-1, 1e1), ha='center', va='center', fontsize=16, color='limegreen')
ell_tform = ScaledTranslation(4.5e-1, 6e0, plt.gca().transScale) + plt.gca().transLimits + plt.gca().transAxes
el2 = patches.Ellipse(xy=(0, 0), width=4e-1, height=3e-1, angle=0, facecolor='limegreen', alpha=.25, edgecolor='none', transform=ell_tform, zorder=-100)

plt.annotate('Alkali\nHalides', (7e-2, 3e0), ha='center', va='center', fontsize=16, color='#FFCE2B')
ell_tform = ScaledTranslation(1e-1, 1, plt.gca().transScale) + plt.gca().transLimits + plt.gca().transAxes
el3 = patches.Ellipse(xy=(0, 0), width=6e-1, height=3e-1, angle=45, facecolor='#FFCE2B', alpha=.25, edgecolor='none', transform=ell_tform, zorder=-100)

plt.annotate('Copper\nHalides', (3.5e-1, 8e-2), ha='center', va='center', fontsize=16, color='darkorange')
ell_tform = ScaledTranslation(1.6e-1, 3.2e-1, plt.gca().transScale) + plt.gca().transLimits + plt.gca().transAxes
el4 = patches.Ellipse(xy=(0, 0), width=4.5e-1, height=8e-1, angle=20, facecolor='darkorange', alpha=.25, edgecolor='none', transform=ell_tform, zorder=-100)

plt.annotate('Sb-chalcohalides', (2e0, 2.5e1), ha='center', va='center', fontsize=16, color='darkviolet')
ell_tform = ScaledTranslation(5.7e-1, 5e0, plt.gca().transScale) + plt.gca().transLimits + plt.gca().transAxes
el5 = patches.Ellipse(xy=(0, 0), width=2e-1, height=1.5e0, angle=-38, facecolor='darkviolet', alpha=.25, edgecolor='none', transform=ell_tform, zorder=-100)

for ellipse in [el1, el2, el3, el4, el5]:
    plt.gca().add_patch(ellipse)

plt.ylabel('RSP = $\\frac{\\varepsilon_{s}}{\\varepsilon_{\infty}}-1$')
plt.xlabel('Microhardness [GPa]')
plt.xscale('log')
plt.yscale('log')
plt.tick_params(direction='in', length=5)
plt.tick_params(direction='in', which='both')
plt.savefig(figures_path+'Figure5.pdf', bbox_inches="tight", pad_inches=0.1)
plt.savefig(figures_path+'Figure5.png', bbox_inches="tight", pad_inches=0.1)
plt.show()

# Classification ML (Figure 4,6)

In [ ]:
cigs_pyrites = [i for i, r in df.iterrows() if r.chemical_formula in ['GaCuSe2', 'InCuSe2']] #Cu(In,Ga)Se2

#Sb chalcohalides
sbs_ch = list(df.loc[df.chemical_formula=='Sb2S3'].index)
sbse_ch = list(df.loc[df.chemical_formula=='Sb2Se3'].index)

#csofcs
csofcs = [i for i, r in df.iterrows() if any([s in r.chemical_formula for s in ['YSZ','La1.']]) or r.chemical_formula in ['La2Mo2O9', 'Ba2YCu3O7']]


dt_df = df.loc[haps + chalcohalides + sbs_ch + sbse_ch + alkali_halides + coinage_halides + cigs_pyrites + csofcs]
dt_df = dt_df.filter(['space_group', 'chemical_formula', 'e_ionic', 'e_electronic', 'ODP'])
for i in csofcs:
    dt_df.at[i, 'ODP'] = df.loc[CSOFCs].ODP.mean()
dt_df['RSP'] = dt_df.e_ionic/dt_df.e_electronic
dt_df = dt_df.filter(['chemical_formula', 'space_group', 'RSP', 'ODP'])

In [ ]:
number = []
for i, r in dt_df.iterrows():
    c = r.chemical_formula
    if i in alkali_halides or i in coinage_halides or i in cigs_pyrites or i in csofcs or c=='Ag3SI' or c.startswith('Li6PS5'):
        number.append(1)
    elif (i in haps and ('Pb' in c or 'Sn' in c or 'Ge' in c)) or (i in chalcohalides
                                                                   and 'Sb' in c and 'In' not in c and 'Cd' not in c) or c=='Sb2S3' or c=='Sb2Se3':
        number.append(2)
    else:
        number.append(None)
dt_df['class'] = number

In [ ]:
dt_df = dt_df.loc[[i for i, r in dt_df.iterrows() if r.chemical_formula!='CsGeBr3' or r.space_group!=221]]
dt_df = dt_df.loc[(np.log10(dt_df.RSP))<4] #no CsSnI3
#both are outliers according to Figure 3b
dt_df = dt_df.dropna()

In [ ]:
X = dt_df.drop(['chemical_formula', 'space_group', 'class'], axis=1)
y = dt_df.filter(['class'], axis=1)

X['RSP'] = np.log10(X['RSP'])
X = X[['RSP', 'ODP']]

In [ ]:
clf = SVC(C=1, kernel='linear', random_state=42).fit(X, y) #training SVM on supervised data

gmm = GaussianMixture(n_components=2, random_state=42, init_params='kmeans').fit(X) #training SVM on unsupervised data
labels = gmm.predict(X)
probs = gmm.predict_proba(X)
means = gmm.means_
covs = gmm.covariances_

In [ ]:
dt_df_pred = dt_df.copy()
dt_df_pred['pred'] = clf.predict(X)

In [ ]:
x_min, x_max = np.log(2e-2), np.log(3e1)
y_min, y_max = -12, 6
xx, yy = np.meshgrid(np.linspace(x_min, x_max, 500),
                     np.linspace(y_min, y_max, 500))

Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)

w = clf.coef_[0]
b = clf.intercept_[0]
m = 1/np.linalg.norm(w)
slope = -w[0] / w[1]
intercept = -b / w[1]

cmap = cm.get_cmap('Set3')
ccmap = colors.ListedColormap(['darkorchid','yellow'])

cf = plt.contourf(xx, yy, Z, cmap=ccmap, alpha=0.5) # plotting the decision boundary

def lin(x, offset):
    return slope*x +intercept + offset/w[1]
    
plt.plot([-2, 2], [lin(k, 0) for k in [-2,2]], linestyle='-', color='black', zorder=50, linewidth=3)
#plotting the margin
plt.plot([-2, 2], [lin(k,1) for k in [-2,2]], linestyle=(0, (5, 10)), color='black', zorder=50, linewidth=1)
plt.plot([-2, 2], [lin(k,-1) for k in [-2,2]], linestyle=(0, (5, 10)), color='black', zorder=50, linewidth=1)


c=['#FFCE2B', 'mediumorchid']
for i in range(2): #plotting the gaussians
    mean = means[i]
    cov = covs[i]
    
    v, w = np.linalg.eigh(cov)  # Get the eigenvalues = variances
    v = np.sqrt(v)  # The radii of the ellipse (std deviations)

    # Angle of the ellipse (from eigenvectors)
    angle = np.arctan2(w[1, 0], w[0, 0])

    ells = []
    for l in [1,3,5]: #1 std, 3 stds and 5 stds
        ell = patches.Ellipse(xy=(mean[0], mean[1]), width=l * v[0], height=l * v[1],
                              angle=np.degrees(angle), edgecolor=None, alpha=0.35, facecolor=c[i], lw=2)

        ells+=[ell]
    
    # Convert log center back to linear for plotting
    for ell in ells:
        plt.gca().add_patch(ell)

plt.scatter(*means[0], c='khaki', s=300, marker='X', edgecolor='k', zorder=300)
plt.scatter(*means[1], c='blueviolet', s=300, marker='X', edgecolor='k', zorder=300)
plt.scatter(X['RSP'], X['ODP'], c=y['class'].tolist(), cmap=colors.ListedColormap(['blueviolet', 'khaki']), s=40, edgecolor='k', zorder=300)

plt.xlabel('RSP = $\\frac{\\varepsilon_{s}}{\\varepsilon_{\\infty}}-1$')
plt.ylabel('ODP [eV]')

major_ticks = [-1,0,1]
major_labels = ['10'+"\u207B"+"\u00B9", '10'+"\u2070", '10'+"\u00B9"]
plt.xticks(major_ticks)
plt.gca().set_xticklabels(major_labels, fontname='Helvetica')

#Set minor ticks: to simulate log scale minor ticks (e.g., 2, 3,...9 between each decade)
minor_ticks = []
for i in range(-2, 2):  # from 10^-2 to 10^1
    for j in range(2, 10):  # 2*10^i, ..., 9*10^i
        minor_ticks.append(np.log10(j) + i)

plt.gca().set_xticks(minor_ticks, minor=True)

plt.annotate('Dynamic\nLattice', (1, 4.5), fontsize=40, c='gold', ha='center', va='center',
             path_effects=[pe.withStroke(linewidth=2, foreground="k"),
                           pe.withSimplePatchShadow(offset=(2.5, -2.5), shadow_rgbFace="gray", alpha=0.4)], zorder=1000)

plt.annotate('Static\nLattice', (-1, -4.6), fontsize=40, c='blueviolet', ha='center', va='center',
             path_effects=[pe.withStroke(linewidth=2, foreground="k"),
                           pe.withSimplePatchShadow(offset=(2.5, -2.5), shadow_rgbFace="gray", alpha=0.4)], zorder=1000)

plt.tick_params(direction='in', length=5)
plt.tick_params(direction='in', which='both')

plt.xlim([-1.4, 1.45])
plt.ylim([-6.1, 5.7])

plt.show()

In [ ]:
slope * np.log10(dt_df.RSP).std()/dt_df.ODP.std() #close to unity

In [ ]:
######### Figure 6 #########
plt.figure(figsize=(10,8))
plt.scatter(np.log10(df.e_ionic/df.e_electronic), df.ODP, color='black', alpha=0.1, marker='o', s=10, edgecolor='none')

x_min, x_max = np.log(2e-2), np.log(3e1)
y_min, y_max = -12, 6
xx, yy = np.meshgrid(np.linspace(x_min, x_max, 500),
                     np.linspace(y_min, y_max, 500))

# Predict over grid
Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)

w = clf.coef_[0]
b = clf.intercept_[0]
m = 1/np.linalg.norm(w)
slope = -w[0] / w[1]
intercept = -b / w[1]

ccmap = colors.ListedColormap(['darkorchid','yellow'])

cf = plt.contourf(xx, yy, Z, cmap=ccmap, alpha=0.25)
cs = ['#FFCE2B', 'mediumorchid']
for i in range(2):
    mean = means[i]
    cov = covs[i]
    
    v, w = np.linalg.eigh(cov)  # Get the eigenvalues = variances
    v = np.sqrt(v)  # The radii of the ellipse (std deviations)

    # Angle of the ellipse (from eigenvectors)
    angle = np.arctan2(w[1, 0], w[0, 0])

    ells = []
    for l in [1,3,5]:
        ell = patches.Ellipse(xy=(mean[0], mean[1]), width=l * v[0], height=l * v[1],
                              angle=np.degrees(angle), edgecolor=c[i], alpha=0.35, fill=False, lw=2)

        ells+=[ell]
    
    # Convert log center back to linear for plotting
    for ell in ells:
        plt.gca().add_patch(ell)

major_ticks = [-1,0,1]
major_labels = ['10'+"\u207B"+"\u00B9", '10'+"\u2070", '10'+"\u00B9"]
plt.xticks(major_ticks)
plt.gca().set_xticklabels(major_labels, fontname='Helvetica')

#Set minor ticks: to simulate log scale minor ticks (e.g., 2, 3,...9 between each decade)
minor_ticks = []
for i in range(-2, 2):  # from 10^-2 to 10^1
    for j in range(2, 10):  # 2*10^i, ..., 9*10^i
        minor_ticks.append(np.log10(j) + i)
plt.gca().set_xticks(minor_ticks, minor=True)

plt.plot([-2, 2], [lin(k, 0) for k in [-2,2]], linestyle='-', color='black', zorder=50, linewidth=3)

#Predicting class II for several materials:

plt.scatter(np.log10(37.66808431/5.688204545), 0.885389827, marker='s', s=100, c='k')
plt.annotate('BiVO$\\mathdefault{_4}$', (np.log10(37.66808431/5.688204545), 0.8), fontsize=15, ha='center', va='top')

plt.scatter(np.log10(39.0433/4.532348), -1.265105, marker='s', s=100, c='k')
plt.annotate('Ba$\\mathdefault{_{2}}$ZrO$\\mathdefault{_{4}}$', (np.log10(39.0433/4.532348), -1.34), fontsize=15, ha='center', va='top')

BHO = df.loc[df.chemical_formula=='Ba2HfO4']
plt.scatter(np.log10(BHO.e_ionic/BHO.e_electronic), BHO.ODP, marker='s', s=100, c='k')
plt.annotate('Ba$\\mathdefault{_{2}}$HfO$\\mathdefault{_{4}}$', (np.log10(BHO.e_ionic/BHO.e_electronic)+0.02, BHO.ODP), fontsize=15, ha='left', va='center')

BHS = df.loc[df.chemical_formula=='Ba2HfS4']
plt.scatter(np.log10(BHS.e_ionic/BHS.e_electronic), BHS.ODP, marker='D', s=100, c='k')
plt.annotate('Ba$\\mathdefault{_{2}}$HfS$\\mathdefault{_{4}}$', (np.log10(BHS.e_ionic/BHS.e_electronic)+0.02, BHS.ODP), fontsize=15, ha='left', va='center')

BZS = df.loc[df.chemical_formula=='Ba2ZrS4']
plt.scatter(np.log10(BZS.e_ionic/BZS.e_electronic), BZS.ODP, marker='D', s=100, c='k')
plt.annotate('Ba$\\mathdefault{_{2}}$ZrS$\\mathdefault{_{4}}$', (np.log10(BZS.e_ionic/BZS.e_electronic)+0.02, BZS.ODP+0.02), fontsize=15, ha='left', va='center')


plt.scatter(np.log10(107.28/6.72), -1.24, marker='*', s=150, c='k')
plt.annotate('AgBiSCl$\\mathdefault{_{2}}$', (np.log10(107.28/6.72), -1.18), ha='center',va='bottom', fontsize=15)


plt.scatter(np.log10(9.344151283/6.5938926), 1.468750083, marker='X', s=100, c='k')
plt.annotate('BaYCuS$\\mathdefault{_{3}}$', (np.log10(9.344151283/6.5938926)+0.02, 1.469), ha='left', fontsize=15, va='center')

bycse = df.loc[df.chemical_formula=='BaYCuSe3']
plt.scatter(np.log10(bycse.e_ionic/bycse.e_electronic), bycse.ODP, marker='X', s=100, c='k')
plt.annotate('BaYCuSe$\\mathdefault{_{3}}$', (np.log10(bycse.e_ionic/bycse.e_electronic)+0.02, bycse.ODP), ha='left', fontsize=15, va='center')

bycte = df.loc[df.chemical_formula=='BaYCuTe3']
plt.scatter(np.log10(bycte.e_ionic/bycte.e_electronic), bycte.ODP, marker='X', s=100, c='k')
plt.annotate('BaYCuTe$\\mathdefault{_{3}}$', (np.log10(bycte.e_ionic/bycte.e_electronic)+0.02, bycte.ODP), ha='left', fontsize=15, va='center')

plt.scatter(np.log10(12.76064066/7.557652533), 1.241565898, marker='X', s=100, c='k')
plt.annotate('BaScCuS$\\mathdefault{_{3}}$', (np.log10(12.76064066/7.557652533)+0.02, 1.241565898), fontsize=15, zorder=1000, ha='left', va='center')


TSI = df.loc[df.chemical_formula=='TlSnI3'].dropna(subset='e_ionic')
plt.scatter(np.log10(TSI.e_ionic/TSI.e_electronic), TSI.ODP, marker='o', s=100, c='k')
plt.annotate('TlSnI$\\mathdefault{_{3}}$', (np.log10(TSI.e_ionic/TSI.e_electronic), TSI.ODP+0.04), fontsize=15, ha='center', va='bottom')

ISC = df.loc[df.chemical_formula=='InSnCl3'].dropna(subset='e_ionic')
plt.scatter(np.log10(ISC.e_ionic/ISC.e_electronic), ISC.ODP, marker='o', s=100, c='k')
plt.annotate('InSnCl$\\mathdefault{_{3}}$', (np.log10(ISC.e_ionic/ISC.e_electronic)*0.97, ISC.ODP), fontsize=15, ha='right', va='center')

TCC = df.loc[df.chemical_formula=='TlCdCl3'].dropna(subset='e_ionic')
plt.scatter(np.log10(TCC.e_ionic/TCC.e_electronic), TCC.ODP, marker='o', s=100, c='k')
plt.annotate('TlCdCl$\\mathdefault{_{3}}$', (np.log10(TCC.e_ionic/TCC.e_electronic)+0.01, TCC.ODP), fontsize=15, ha='left', va='center')


PbS = df.loc[df.chemical_formula=='PbS'].dropna(subset='e_ionic')
plt.scatter(np.log10(PbS.e_ionic/PbS.e_electronic), PbS.ODP, marker='^', s=100, c='k')
plt.annotate('PbS', (np.log10(PbS.e_ionic/PbS.e_electronic), PbS.ODP+0.04), ha='center', fontsize=15, va='bottom')

PbSe = df.loc[df.chemical_formula=='PbSe'].dropna(subset='e_ionic')
plt.scatter(np.log10(PbSe.e_ionic/PbSe.e_electronic), PbSe.ODP, marker='^', s=100, c='k')
plt.annotate('PbSe', (np.log10(PbSe.e_ionic/PbSe.e_electronic), PbSe.ODP+0.04), ha='center', fontsize=15, va='bottom')

PbTe = df.loc[df.chemical_formula=='TePb'].dropna(subset='e_ionic')
plt.scatter(np.log10(PbTe.e_ionic/PbTe.e_electronic), PbTe.ODP, marker='^', s=100, c='k')
plt.annotate('PbTe', (np.log10(PbTe.e_ionic/PbTe.e_electronic), PbTe.ODP+0.04), ha='center', fontsize=15, va='bottom')

plt.scatter(np.log10(29.839044/4.712965), -1.085729, marker='^', s=100, c='k')
plt.annotate('PbCl$\\mathdefault{_{2}}$', (np.log10(29.839044/4.712965)+0.015, -1.085729), fontsize=15, zorder=1000, ha='left', va='center')

PbI2 = df.loc[df.chemical_formula=='PbI2'].loc[df.space_group==164]
pi_odp = PbI2.ODP.mean()
pi_rsp = np.log10(PbI2.e_ionic.mean()/PbI2.e_electronic.mean())
plt.scatter(pi_rsp, pi_odp, marker='^', s=100, c='k')
plt.annotate('PbI$\mathdefault{_{2}}$', (pi_rsp+0.015, pi_odp), fontsize=15, zorder=1000, ha='left', va='center')

BZS = df.loc[df.chemical_formula=='BaZrS3']
plt.scatter(np.log10(BZS.e_ionic/BZS.e_electronic), BZS.ODP, marker='D', s=100, c='k')
plt.annotate('BaZrS$\\mathdefault{_{3}}$', (np.log10(BZS.e_ionic/BZS.e_electronic), BZS.ODP-0.2), fontsize=15, zorder=1000, ha='center', va='center')


plt.plot([-2, 2], [lin(k,1) for k in [-2,2]], linestyle=(0, (5, 10)), color='black', zorder=50, linewidth=1)
plt.plot([-2, 2], [lin(k,-1) for k in [-2,2]], linestyle=(0, (5, 10)), color='black', zorder=50, linewidth=1)

plt.xlabel('RSP = $\\frac{\\varepsilon_{s}}{\\varepsilon_{\\infty}}-1$')
plt.ylabel('ODP [eV]')
plt.tick_params(direction='in', length=5)
plt.tick_params(direction='in', which='both')

plt.xlim([np.log10(9e-1), np.log10(3e1)])
plt.ylim([-1.8, 3.9])
plt.scatter(*means[0], c='gold', s=300, marker='X', edgecolor='k', zorder=300)
plt.show()